In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
#tf.enable_eager_execution()

from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd

import keras
import keras.backend
from keras import layers
from keras import models
import keras.utils

# Helper libraries
import numpy as np
from scipy import stats,special

from matplotlib.colors import LogNorm
from matplotlib import gridspec
import math
import time
import h5py
import sklearn
from sklearn.utils import shuffle
from awkward import JaggedArray, MaskedArray

# innvestigate for lrp
import innvestigate
import innvestigate.utils as iutils
from sklearn.metrics import roc_curve, auc

import matplotlib.pyplot as plt
import matplotlib.cm

import os
import datetime




import sys
sys.path.append('/mnt/users/morris35/results/CNNsimple/Python')

from networkBuilder import *

print('TensorFlow Version {}'.format(tf.__version__))

In [ ]:
params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'font.family':'serif',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large',
         'figure.facecolor':'white',
         'axes.grid':True,
         'grid.alpha':1.0}
plt.rcParams.update(params)
plt.style.context('default')

## Load Data

In [ ]:
test_sig = np.load('/mnt/data/ml/PreProcessing/ShowJets_test_Zbb_exts_msd50to150_v2.npz')
test_bkg = np.load('/mnt/data/ml/PreProcessing/ShowJets_test_QCD_exts_msd50to150_v2.npz')

In [ ]:
options = [
    'Particle List Only',
    'Particle List Only no flavor',
    'Particle List + all XAUGS',
    'Particle List + XAUGS no flavor',
    'Particle List + 5 XAUGS',
    'XAUGS only',
    'XAUGS only no flavor',
]

option = 'Particle List Only'
model_name = '1DCNN_plist_1.h5'

## Load Model and Get Test Data arrays

In [ ]:
best_model = keras.models.load_model('model/'+model_name)



In [ ]:
# rename dense layers for softmax layer removal function later

layer_i = 1

for layer in best_model.layers:
    if('dense' in layer.name):
        layer.name = 'renamed_dense_layer'+str(layer_i)
        layer_i += 1


In [ ]:
top_five_xaugs = [
    'jetMassSD',
    'tau3_b10',
    'tau3_sd_b05',
    'dxy_max',
    'dz_max',
]

top_xaugs = []

if('+ 5 XAUGS' in option):
    top_xaugs = top_five_xaugs

In [ ]:
def getInputInfo(option, top_xaugs = []):
    
    
    
    particleInfo = True
    xaugs = True
    flavor = True

    if('Particle List Only'==option):
        particleInfo = True
        xaugs = False
        flavor = False
        
    elif('Particle List + all XAUGS'==option):
        particleInfo = True
        xaugs = True
        flavor = True
        
    elif('Particle List + XAUGS no flavor'==option):
        particleInfo = True
        xaugs = True
        flavor = True
        
    elif('Particle List + 5 XAUGS'==option):
        particleInfo = True
        xaugs = False
        flavor = False
        
    elif('XAUGS only'==option):
        particleInfo = False
        xaugs = True
        flavor = True
    
    elif('XAUGS only no flavor'==option):
        particleInfo = False
        xaugs = True
        flavor = False
        
        
    features = get_feat(xaugs=xaugs, flav=flavor, particleInfo=particleInfo)
    
    features += top_xaugs

    nXvar = 0
    if(xaugs): nXvar = 29
    if(flavor): nXvar += 7    
    nXvar += len(top_xaugs)
        
        
    print('option = {0}'.format(option))
    print('particleInfo = {0}'.format(particleInfo))
    print('xaugs = {0}'.format(xaugs))
    print('flavor = {0}'.format(flavor))
    print('nXvar = {0}'.format(nXvar))

    
        
    return features, nXvar, particleInfo

In [ ]:
features, nXvar, particleInfo = getInputInfo(option, top_xaugs = top_xaugs)

In [ ]:
totalVar = len(features)
for i, feat in enumerate(features):
    print(i, feat)

In [ ]:
X_test,  Y_test  = build_XY(features, ['labels'], test_sig,  test_bkg)

In [ ]:
# delete loaded npz files
del test_sig
del test_bkg

In [ ]:
pred = best_model.predict(X_test)

## Functions

In [ ]:
def printTime(delta, words='Elapsed Time'):
    time_string = words + ': ' +'{0:0.1f} h {1:0.1f} min {2:0.2f} sec'.format(delta // 3600, delta % 3600 // 60, delta % 3600 % 60)
    print(time_string) 

In [ ]:
def get_lrp_score(test_inputs, model, nXvar, totalVar, batch=20):
    
    # model loaded from model = keras.models.load_model(model_name)
    # nXvar = number of expert features (xaugs)
    # totalVar = total number of features
    # batch = number of constituent particles per event
    
    lrp_toc = time.time()
    
    # strip model of softmax layer
    best_model_wo_softmax = innvestigate.utils.keras.graph.model_wo_softmax(model)
     
    # build lrp analyzer using Preset A: LRP_alpha1beta0 for conv layers and LRP_epsilon for dense layers
    lrp_analyzer = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPSequentialPresetA(best_model_wo_softmax)
   
    # run lrp analysis in chunks so that kernel does not die
    nElem = 10000
    batchsize = test_inputs[0].shape[0]
    slices = batchsize // nElem
    size_from_loop = batchsize - (batchsize % nElem)
    
    print()
    print('Running LRP Analysis')
    print('Total Events: {0:0.0f}'.format(batchsize))
    print('Split into chunk size: {0:0.0f}'.format(nElem))
    print()
    
    lrp_list = []
    

    # get lrp scores from all chunks except last chunk
    for i in range(slices):

        lrp_i = lrp_analyzer.analyze([element[nElem*i:nElem*(i+1)] for element in test_inputs])
        lrp_list.append(lrp_i)
        del lrp_i

    
    # get lrp scores from last chunk
    lrp_list_last = lrp_analyzer.analyze([element[size_from_loop:] for element in test_inputs])
    
    # combine chunks into separate arrays for particle list features (lrp_plist) and xaug features (lrp_xaugs)
    lrp_plist_list = []
    lrp_xaugs_list = []
    
    lrp_plist_list_last = []
    lrp_xaugs_list_last = []


    # append lrp scores to lrp_plist or lrp_xaugs
    for i in range(totalVar):
        for s in range(slices):
            if(i < (totalVar-nXvar)):
                lrp_plist_list.append(lrp_list[s][i])
                    
            else:
                lrp_xaugs_list.append(lrp_list[s][i])
                
    # append lrp scores from last chunk         
    for i in range(totalVar):
        if(i < (totalVar-nXvar)):
            lrp_plist_list_last.append(lrp_list_last[i])

        else:
            lrp_xaugs_list_last.append(lrp_list_last[i])
                
                
    # copy to arrays           
    lrp_plist_first = np.array(lrp_plist_list).reshape(totalVar-nXvar, size_from_loop, batch, 1)
    lrp_xaugs_first = np.array(lrp_xaugs_list).reshape(nXvar, size_from_loop, 1)
    
    lrp_plist_last = np.array(lrp_plist_list_last).reshape(totalVar-nXvar,(batchsize % nElem),batch,1)
    lrp_xaugs_last = np.array(lrp_xaugs_list_last).reshape(nXvar,(batchsize % nElem),1)
                
    
    # append last chunk to lrp scores arrays
    lrp_plist = np.append(lrp_plist_first, lrp_plist_last, axis=1)
    lrp_xaugs = np.append(lrp_xaugs_first, lrp_xaugs_last, axis=1)

    del lrp_plist_list
    del lrp_xaugs_list
    del lrp_plist_list_last
    del lrp_xaugs_list_last
    del lrp_list
    
    lrp_tic = time.time()
    
    print()
    printTime(lrp_tic - lrp_toc, 'LRP Analysis Time')
    
    return lrp_plist, lrp_xaugs
    

In [ ]:
def get_normalized_lrp_score(lrp_plist, lrp_xaugs):

    lrp_plist_norm = np.zeros_like(lrp_plist)
    lrp_xaugs_norm = np.zeros_like(lrp_xaugs)
    
    if(len(lrp_plist > 0)):
        for i in range(lrp_plist.shape[1]):

            maxval = np.max(abs(lrp_plist[:,i].flatten()))
            if(maxval < 1e-6): maxval = 1.
            lrp_plist_norm[:,i] = lrp_plist[:,i] / maxval

    if(len(lrp_xaugs > 0)):   
        for i in range(analysis_a1b0_full_xv_norm.shape[1]):

            maxval = np.max(abs(lrp_xaugs[:,i].flatten()))
            lrp_xaugs_norm[:,i] = lrp_xaugs[:,i] / maxval
    
    
    return lrp_plist_norm, lrp_xaugs_norm
    
    
    
    

In [ ]:
def make_profile_data(x,values,bins):
    bin_mean, bin_edges, _ = stats.binned_statistic(x,
                                                    values,
                                                    statistic = 'mean',
                                                    bins = bins,
                                                    range = (0,1))
    bin_std, _, _ = stats.binned_statistic(x,
                                           values,
                                           statistic = 'std',
                                           bins = bins,
                                           range = (0,1))
    bin_count, _, _ = stats.binned_statistic(x,
                                             x,
                                             statistic = 'count',
                                             bins = bins,
                                             range = (0,1))
    bin_width = (bin_edges[1] - bin_edges[0])
    bin_centers = bin_edges[1:] - bin_width/2
    return bin_mean, bin_std, bin_count, bin_edges, bin_centers, bin_width

In [ ]:
def Draw_Profile(X_test, Y_test, pred, LRP_norm, feat, bins, histomax, relmin, relmax):
    ii = features.index(feat)
    
#     print(ii)
#     print(features[ii])
#     print(LRP_norm[ii])
#     ind_positive = np.argwhere((pred[:,0,1]>confidence_cut)).flatten()
#     ind_negative = np.argwhere((pred[:,0,0]>confidence_cut)).flatten()

    ind_positive = (Y_test[0][:,1]==1)
    ind_negative = (Y_test[0][:,1]==0)

    LRP_p = LRP_norm[ii][ind_positive].flatten()
    LRP_n = LRP_norm[ii][ind_negative].flatten()
    X_p = X_test[ii][ind_positive].flatten()
    X_n = X_test[ii][ind_negative].flatten()
    
    bin_mean_p, bin_std_p, bin_count_p, bin_edges, bin_centers, bin_width = make_profile_data(X_p, LRP_p, bins)
    bin_mean_n, bin_std_n, bin_count_n, _, _, _  = make_profile_data(X_n, LRP_n, bins)
    
    bin_mean_p[bin_count_p < 50] = np.nan
    bin_std_p[bin_count_p < 50] = 0
    
    bin_mean_n[bin_count_n < 50] = np.nan
    bin_std_n[bin_count_n < 50] = 0

    fig = plt.figure(figsize=(6,7))
    gs = gridspec.GridSpec(3, 1) 
    gs.update(wspace=0.025, hspace=0.1)
    
    ax0 = plt.subplot(gs[:2,:])
    ax0.hist(X_p.flatten(),
                bins = bins,
                histtype = 'step',
#                normed=False,
                weights = np.ones(len(X_p.flatten()))/len(X_p.flatten()),
                fill = True,
                alpha = 0.55,
                label = "Signal",
                log = False,
#                density = True,
                range = [0,1],
                hatch='/',
                edgecolor='k'
                );
    ax0.hist(X_n.flatten(),
                bins = bins,
                histtype = 'step',
#                normed=False,
                weights = np.ones(len(X_n.flatten()))/len(X_n.flatten()),
                fill = True,
                alpha = 0.55,
                label = 'Background',
                log = False,
#                density = True,
                range = [0,1],
                hatch = '\\',
                edgecolor='k'
                );
    l = ax0.legend();
#    ax0.grid()
    ax0.set_xlim(0,1)
    ax0.set_ylim(0,histomax)

    ax0.set_ylabel('Fraction')

    ax1 = plt.subplot(gs[2:,:])
    ax1.xaxis.set_ticks(np.arange(0, 1.2, 0.2))
    plt.setp(ax0.get_xticklabels(), visible=False)
    ax1.plot(bin_centers,
             bin_mean_p,
             color = 'C0',
             label = "Signal")
    ax1.plot(bin_centers,
             bin_mean_n,
             color = 'C1',
             label = "Background")
    ax1.fill_between(bin_centers,
                     bin_mean_p - bin_std_p,
                     bin_mean_p + bin_std_p,
                     color = 'C0',
                     alpha = 0.2)
    ax1.fill_between(bin_centers,
                     bin_mean_n - bin_std_n,
                     bin_mean_n + bin_std_n,
                     color = 'C1',
                     alpha = 0.2)
    ax1.grid()
    ax1.set_ylim(relmin,relmax)
    ax1.set_xlim(0, 1)
    ax1.set_ylabel('Relevance')
    ax1.set_xlabel('Rescaled ' + feature_names[feat])

### Get LRP Scores

In [ ]:
lrp_plist, lrp_xaugs = get_lrp_score(X_test, best_model, nXvar, totalVar)

### Get Normalized LRP Scores

In [ ]:
lrp_plist_norm, lrp_xaugs_norm, = get_normalized_lrp_score(lrp_plist, lrp_xaugs)

## Plots

In [ ]:
feature_names={'jetPt':r'$p_{T,jet}$',
               'jetEta':r'$\eta(jet)$',
               'jetPhi':r'$\phi(jet)$',
               'jetMass':r'$m_{jet}$',
               'jetMassSD':r'$m_{jet,sd}$',
               'tau1_b05':r'$\tau_{1}^{(0.5)}$',
               'tau2_b05':r'$\tau_{2}^{(0.5)}$',
               'tau3_b05':r'$\tau_{3}^{(0.5)}$',
               'tau1_sd_b05':r'$\tau_{1,sd}^{(0.5)}$',
               'tau2_sd_b05':r'$\tau_{2,sd}^{(0.5)}$',
               'tau3_sd_b05':r'$\tau_{3,sd}^{(0.5)}$',
               'tau1_b10':r'$\tau_{1}^{(1)}$',
               'tau2_b10':r'$\tau_{2}^{(1)}$',
               'tau3_b10':r'$\tau_{3}^{(1)}$',
               'tau1_sd_b10':r'$\tau_{1,sd}^{(1)}$',
               'tau2_sd_b10':r'$\tau_{2,sd}^{(1)}$',
               'tau3_sd_b10':r'$\tau_{3,sd}^{(1)}$',
               'tau1_b15':r'$\tau_{1}^{(1.5)}$',
               'tau2_b15':r'$\tau_{2}^{(1.5)}$',
               'tau3_b15':r'$\tau_{3}^{(1.5)}$',
               'tau1_sd_b15':r'$\tau_{1,sd}^{(1.5)}$',
               'tau2_sd_b15':r'$\tau_{2,sd}^{(1.5)}$',
               'tau3_sd_b15':r'$\tau_{3,sd}^{(1.5)}$',
               'tau1_b20':r'$\tau_{1}^{(2)}$',
               'tau2_b20':r'$\tau_{2}^{(2)}$',
               'tau3_b20':r'$\tau_{3}^{(2)}$',
               'tau1_sd_b20':r'$\tau_{1,sd}^{(2)}$',
               'tau2_sd_b20':r'$\tau_{2,sd}^{(2)}$',
               'tau3_sd_b20':r'$\tau_{3,sd}^{(2)}$',
               'chMult':r'$N_{ch}$',
               'neutMult':r'$N_{neut}$',
               'phoMult':r'$N_{\gamma}$',
               'eleMult':r'$N_{e}$',
               'muMult':r'$N_{\mu}$',
               'jetpull':r'$\phi_{pull}$',
               'beta3':r'$\beta_{3}$',
               'beta3_sd':r'$\beta_{3}^{g}$',
               'tau21':r'$\tau_{2}^{(1)} / \tau_{1}^{(1)}$',
               'deltaR_subjets':r'$\Delta_r$',
               'z':r'$z$',
               'dxy_max':r'$d_{xy,max}$',
               'dz_max':r'$d_{z,max}$',
               'dxy':'$d_{xy}$',
               'dz':'$d_{z}$',
              }

In [ ]:
relmin = -1
relmax = 1
bins = 50
histomax = 0.41
feat = 'dxy'


Draw_Profile(X_test, Y_test, pred, lrp_plist_norm, feat, bins, histomax, relmin, relmax)

In [ ]:
relmin = -1
relmax = 1
bins = 50
histomax = 0.41
feat = 'dz'


Draw_Profile(X_test, Y_test, pred, lrp_plist_norm, feat, bins, histomax, relmin, relmax)